In [50]:
pip install websocket-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
pip install python-binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.5 MB/s 
     |████████████████████████████████| 288 kB 18.3 MB/s 
     |████████████████████████████████| 112 kB 35.9 MB/s 
     |████████████████████████████████| 749 kB 32.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [57]:
import os
import websocket, json

import datetime as dt
from datetime import datetime

import requests
from binance.client import Client
import pandas as pd

import matplotlib.pyplot as plt

Get 1000 historical candles

In [67]:
root_url = 'https://api.binance.com/api/v1/klines'
def get_bars(symbol, interval = '1m'):
   url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&limit=1000'
   data = json.loads(requests.get(url).text)
   df = pd.DataFrame(data)
   df.columns = ['open_time',
                 'o', 'h', 'l', 'c', 'v',
                 'close_time', 'qav', 'num_trades',
                 'taker_base_vol', 'taker_quote_vol', 'ignore']
   df.index = [dt.datetime.fromtimestamp(x//1000.0) for x in df.close_time]
   return df
btcusdt = get_bars('BTCUSDT')

df0=pd.DataFrame(btcusdt)
print(df0)
df0.to_csv('_btcusdt.csv')

                         open_time               o               h  \
2022-08-15 03:56:59  1660535760000  24858.38000000  24867.07000000   
2022-08-15 03:57:59  1660535820000  24856.35000000  24870.00000000   
2022-08-15 03:58:59  1660535880000  24856.86000000  24882.92000000   
2022-08-15 03:59:59  1660535940000  24867.40000000  24888.00000000   
2022-08-15 04:00:59  1660536000000  24866.12000000  24888.00000000   
...                            ...             ...             ...   
2022-08-15 20:31:59  1660595460000  24079.98000000  24087.32000000   
2022-08-15 20:32:59  1660595520000  24085.77000000  24094.04000000   
2022-08-15 20:33:59  1660595580000  24071.07000000  24072.05000000   
2022-08-15 20:34:59  1660595640000  24039.40000000  24042.52000000   
2022-08-15 20:35:59  1660595700000  24041.78000000  24043.81000000   

                                  l               c             v  \
2022-08-15 03:56:59  24850.00000000  24856.35000000  129.04367000   
2022-08-15 03:57:59  

Get real time candle

In [3]:
cc = 'btcusd'
interval = '1m'


socket = f'wss://stream.binance.com:9443/ws/{cc}t@kline_{interval}'

In [4]:
def on_message(ws, message):
    print(message)

def on_close(ws, close_status_code, close_msg):
    print("Connection closed")

ws = websocket.WebSocketApp(socket,on_message=on_message,on_close=on_close)

In [5]:
ws.run_forever()

{"e":"kline","E":1660587335755,"s":"BTCUSDT","k":{"t":1660587300000,"T":1660587359999,"s":"BTCUSDT","i":"1m","f":1635114536,"L":1635115976,"o":"24074.52000000","c":"24077.40000000","h":"24079.82000000","l":"24073.85000000","v":"32.17646000","n":1441,"x":false,"q":"774708.52947670","V":"17.02573000","Q":"409937.50791540","B":"0"}}
{"e":"kline","E":1660587337992,"s":"BTCUSDT","k":{"t":1660587300000,"T":1660587359999,"s":"BTCUSDT","i":"1m","f":1635114536,"L":1635116120,"o":"24074.52000000","c":"24080.02000000","h":"24081.91000000","l":"24073.85000000","v":"36.80407000","n":1585,"x":false,"q":"886141.05516610","V":"19.74123000","Q":"475324.90231320","B":"0"}}
{"e":"kline","E":1660587339995,"s":"BTCUSDT","k":{"t":1660587300000,"T":1660587359999,"s":"BTCUSDT","i":"1m","f":1635114536,"L":1635116372,"o":"24074.52000000","c":"24077.05000000","h":"24081.91000000","l":"24072.81000000","v":"49.09462000","n":1837,"x":false,"q":"1182045.26437570","V":"24.90094000","Q":"599546.20148540","B":"0"}}
{"e

ERROR:websocket:error from callback <function on_close at 0x7f1fa89b3d40>: on_close() takes 1 positional argument but 3 were given


False

In [43]:
dates, closes, highs, lows = [], [], [], []

In [46]:
#candle

def on_message(ws, message):
    json_message = json.loads(message)
    candle = json_message['k']
    is_candle_closed = candle['x']
    time = candle['T']
    close = candle['c']
    high = candle['h']
    low = candle['l']
    vol = candle['v']

    if is_candle_closed:
      date = datetime.fromtimestamp(float(time)//1000.0).date().isoformat()
      dates.append(date)
      closes.append(float(close))
      highs.append(float(high))
      lows.append(float(low))
    
      print(dates)
      print(closes)
      print(highs)
      print(lows)
    
    df= pd.DataFrame(closes)
    print(df)
    
def on_close(ws, close_status_code, close_msg):
    print("Connection closed")

ws = websocket.WebSocketApp(socket, on_message = on_message, on_close = on_close)

In [47]:
ws.run_forever()

['2022-08-15']
[24111.78]
[24116.29]
[24106.08]
['2022-08-15', '2022-08-15']
[24111.78, 24108.82]
[24116.29, 24116.4]
[24106.08, 24108.54]
Connection closed


False